In [2]:
import torch as th 
from torch_geometric.data import Data, Batch
from torch_geometric.loader import DataLoader

from model import Generator, Discriminator, gradient_penalty

import numpy as np

In [3]:
""" 
Import Data
"""
testing_file = "./dump/graph-data-examples/2.pickle"


""" 
Categories as a dictionary with their indices for onehot encoding
"""
CATEGORY_DICT = {
 'Remaining': 0,
 'Bathroom': 1,
 'Kitchen-Dining': 2,
 'Bedroom': 3,
 'Corridor': 4,
 'Stairs-Ramp': 5,
 'Outdoor-Area': 6,
 'Living-Room': 7,
 'Basement': 8,
 'Office': 9,
 'Garage': 10,
 'Warehouse-Logistics': 11,
 'Meeting-Salesroom': 12
}

In [4]:
loaded_data = th.load(testing_file)
print(loaded_data)
print(loaded_data.category) 
print(type(loaded_data.geometry))

Data(edge_index=[2, 72], geometry=[42], category=[42], centroid=[42, 2], connectivity=[72], door-geometry=[72], walls=[140], num_nodes=42)
['Bedroom', 'Remaining', 'Bathroom', 'Bedroom', 'Bathroom', 'Kitchen-Dining', 'Bedroom', 'Living-Room', 'Corridor', 'Corridor', 'Bedroom', 'Bedroom', 'Corridor', 'Kitchen-Dining', 'Remaining', 'Bathroom', 'Bedroom', 'Bathroom', 'Remaining', 'Remaining', 'Kitchen-Dining', 'Living-Room', 'Corridor', 'Bedroom', 'Bedroom', 'Remaining', 'Corridor', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Remaining', 'Stairs-Ramp']
<class 'list'>


In [5]:
""" 
Hyperparameters
"""

MAX_POLYGONS = 30

# Optimizer params
g_lr = 0.001 
d_lr = 0.001
b1 = 0.5 
b2 = 0.999  

# WGAN params
N_critic = 5            # nr of times to train discriminator more
lambda_gp = 10          # gradient penalty hyperpraram

# Training params
MAX_EPOCHS = 500
BATCH_SIZE = 16

# Network parameters
NOISE_SIZE = 128
HIDDEN_GENERATOR = [128, 128, 128]
OUTPUT_GENERATOR = MAX_POLYGONS * 2             # we want to output at most this many polygons per node (note [x1...y1...] format)

HIDDEN_DISCRIMINATOR = [128, 128, 128]

In [6]:
def onehot(categories):
    """ 
    category: must match CATEGORIES as string
    """
    onehot_encoding = th.zeros((len(categories), len(CATEGORY_DICT)))
    for i, cat in enumerate(categories):
        onehot_encoding[i][CATEGORY_DICT[cat]] = 1
    return onehot_encoding

In [7]:
generator_input_data = []
discriminator_input_data = []

def convert_data(data):
    # Construct generator data    
    onehot_categories = onehot(data.category)
    generator_data = Data(x=onehot_categories, edge_index=data.edge_index)

    # Construct discriminator data
    padded_geometry = th.zeros((len(data.geometry), MAX_POLYGONS * 2), dtype=th.float)
    for i, polygon in enumerate(data.geometry):
        polygon = th.FloatTensor(polygon)
        padded_geometry[i, :polygon.shape[0]] = polygon[:, 0]
        padded_geometry[i, MAX_POLYGONS: MAX_POLYGONS + polygon.shape[0]] = polygon[:, 1]

    discriminator_data = Data(x=padded_geometry, edge_index=data.edge_index)

    return generator_data, discriminator_data

generator_data, discriminator_data = convert_data(loaded_data)

In [8]:
""" 
Data looks as follows:
Generator_data:
x = (N nr. of nodes, onehot of categories)

Discriminator data:
x = (N nr. of nodes, coordinates)
where coordinates looks like: [x1, ..., x30, y1, ..., y30]
and with 0 padding if (xi, yi) doesn't exist
"""
generator_data, discriminator_data = convert_data(loaded_data)

print(generator_data)
print(discriminator_data)    

Data(x=[42, 13], edge_index=[2, 72])
Data(x=[42, 60], edge_index=[2, 72])


In [9]:
""" 
Model Definitions
"""
generator = Generator(input_dim=NOISE_SIZE + len(CATEGORY_DICT), 
                      output_dim=OUTPUT_GENERATOR, 
                      hidden_dims=HIDDEN_GENERATOR)

discriminator = Discriminator(input_dim=OUTPUT_GENERATOR, 
                              hidden_dims=HIDDEN_DISCRIMINATOR)

print(generator.module_list)

ModuleList(
  (0): TAGConv(141, 128, K=3)
  (1): TAGConv(128, 128, K=3)
  (2): TAGConv(128, 60, K=3)
)


In [10]:
# Optimizers
optimizer_G = th.optim.Adam(generator.parameters(), lr=g_lr, betas=(b1, b2)) 
optimizer_D = th.optim.Adam(discriminator.parameters(), lr=d_lr, betas=(b1, b2))

In [11]:
# Main training loop
def train(generator, discriminator, optimizer_g, optimizer_d, data_loader):
    for epoch in range(MAX_EPOCHS):
        # real == batch (confusing naming I know...)
        for real in data_loader:
            # Create new data object with noise and same edge_index 
            for i in range(N_critic):
                noise_batch = []

                # Create n_batch number of noise_vectors and batch it
                # We must create a noise vector with the corresponding graph in the actual data
                for batch_data in real:
                    noise = th.randn(NOISE_SIZE)
                    noise = Data(x=noise, edge_index=batch_data.edge_indices)
                    noise_batch.append(noise)

                noise_batch = Batch.from_data_list(noise_batch)

                # Input noise_data into generator
                global fake 
                fake = Generator(noise_batch)

                # Generator output is a tensor of dimensionality: (sum of all nodes in batch, output_features)
                # We must turn this into appropriate (batch) input for the discriminator
                splits = [batch_data.shape[0] for batch_data in real]       # nr of nodes per graph
                fake_batch = th.split(fake, splits, dim=0)                  # split stacked tensor fake into appropriate batches
                fake = Batch.from_data_list(fake_batch)

                discriminator_fake = discriminator(fake).reshape(-1)        # discriminator scores for fakes
                discriminator_real = discriminator(real).reshape(-1)        # discriminator scores for reals
                gp = gradient_penalty(discriminator, real, fake)

                # Discriminator loss and train
                loss_discriminator = -(th.mean(discriminator_real) - th.mean(discriminator_fake)) + lambda_gp * gp
                discriminator.zero_grad() 
                loss_discriminator.backward() 
                optimizer_d.step()

            # Generator loss and train
            output = discriminator(fake).reshape(-1)        # discriminator scores for fake
            loss_generator = -th.mean(output)               # loss for genereator = the discriminators' judgement
                                                            # higher score = better
            generator.zero_grad()
            loss_generator.backward()
            optimizer_g.step()
    
        # TODO: Evaluation and logging code??

In [14]:
print(generator_data)
print(discriminator_data)

# Generator testing
noise_vector = th.randn(generator_data.x.shape[0], NOISE_SIZE)
generator_input = th.concat((noise_vector, generator_data.x), dim=1)
print(generator_input.shape)

generator_input = Data(x=generator_input, edge_index=generator_data.edge_index)
generator_out = generator(generator_input)

print(generator_out.shape)

# Discriminator testing with Generator output
discriminator_input = Data(x=generator_out, edge_index=discriminator_data.edge_index)
discriminator_out = discriminator(discriminator_input)
print(discriminator_out.shape)

# Discriminator testing with discriminator input (ground truth)
discriminator_out = discriminator(discriminator_data)
print(discriminator_out.shape)

Data(x=[42, 13], edge_index=[2, 72])
Data(x=[42, 60], edge_index=[2, 72])
torch.Size([42, 141])
torch.Size([42, 60])
torch.Size([1, 1])
torch.Size([1, 1])
